In [9]:
import sys
import os
from pathlib import Path

IN_COLAB = False

# if in colab, make colab setup
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    IN_COLAB = True
    ZIP_PATH = "/content/drive/MyDrive/embedded_ml_data/VOCdevkit.zip"
    FILENAME = "dev-notebook.ipynb"
    BRANCH = "michael"
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    !pip install -r requirements.txt

    # setup data
    !cp  $ZIP_PATH data/
    %pushd data
    !unzip -qq VOCdevkit.zip
    %popd

DATA_PATH = "data/"

# sys.path.append('../')

Prepare Tensorboard logger

In [13]:
import lightning as pl
from os import makedirs

LOGDIR='logs'
tensorboard = pl.pytorch.loggers.TensorBoardLogger(save_dir=LOGDIR, default_hp_metric=True)

try:
    os.makedirs('./'+LOGDIR+'/lightning_logs')
except FileExistsError:
    # directory already exists
    pass


%load_ext tensorboard
%tensorboard --logdir $LOGDIR+'lightning_logs'
  

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 112020), started 0:06:51 ago. (Use '!kill 112020' to kill it.)

Prepare model

In [8]:
from models.tinyyolov2 import TinyYoloV2PersonOnly
import lightning as pl

from utils.dataloader import VOCDataModule

trainer = pl.Trainer(
    max_epochs=30,
    auto_scale_batch_size="binsearch",
    accelerator="auto",
    logger=tensorboard,
    log_every_n_steps=1,
    fast_dev_run=not IN_COLAB,
)


model = TinyYoloV2PersonOnly()
model.load_pt_from_disk(DATA_PATH + "/voc_pretrained.pt")

data = VOCDataModule(person_only=True)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


Tune and train model

In [7]:
trainer.tune(model, data)
trainer.fit(model, data)
# trainer.test(model, data)


c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\lightning\pytorch\tuner\batch_size_scaling.py:38: UserWarning: Skipping batch size scaler since `fast_dev_run` is enabled.
  rank_zero_warn("Skipping batch size scaler since `fast_dev_run` is enabled.")

   | Name  | Type            | Params
-------------------------------------------
0  | loss  | YoloLoss        | 0     
1  | pad   | ReflectionPad2d | 0     
2  | conv1 | Conv2d          | 432   
3  | bn1   | BatchNorm2d     | 32    
4  | conv2 | Conv2d          | 4.6 K 
5  | bn2   | BatchNorm2d     | 64    
6  | conv3 | Conv2d          | 18.4 K
7  | bn3   | BatchNorm2d     | 128   
8  | conv4 | Conv2d          | 73.7 K
9  | bn4   | BatchNorm2d     | 256   
10 | conv5 | Conv2d          | 294 K 
11 | bn5   | BatchNorm2d     | 512   
12 | conv6 | Conv2d          | 1.2 M 
13 | bn6   | BatchNorm2d     | 1.0 K 
14 | conv7 | Conv2d          | 4.7 M 
15 | bn7   | BatchNorm2d     | 2.0 K 
16 | conv8 | Conv2d          | 9.4 M 
17 | 

Training: 0it [00:00, ?it/s]

c:\Users\Michael\miniconda3\envs\embedded-ml\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:
from utils.ap import precision_recall_levels, ap, display_roc
from utils.yolo import nms, filter_boxes

import torch
import numpy as np
import tqdm


test_loader = VOCDataLoaderPerson(train=False, batch_size=1, data_path=DATA_PATH, n_limit=350)

test_precision = []
test_recall = []

for inputs, targets in tqdm.tqdm(test_loader, total=350):
    with torch.no_grad():
        outputs = model(inputs)

    #The right threshold values can be adjusted for the target application
    outputs = filter_boxes(outputs, 0.0)
    outputs = nms(outputs, 0.5)
    outputs = torch.tensor(np.array(outputs))

    precision, recall = precision_recall_levels(targets[0], outputs[0])
    test_precision.append(precision)
    test_recall.append(recall)

avg_precision = ap(test_precision, test_recall)
print("average precision: ", avg_precision)
display_roc(test_precision, test_recall)